# Purposes of this document

the goal of this jupyter is to run the BMIloop in steps  
we can  
1. understand what is happening. 
2. develop new components  

along these reasons,  
we will clearly point out where the components are  
we focus on how information is transmitted between components  

note all of this is in the branch sijia as of 2021 Jan


# setting up the simulation components

this section largely copyied and pasted from   
bmi3d-sijia(branch)-bulti_in_experiemnts
https://github.com/sijia66/brain-python-interface/blob/master/built_in_tasks/sim_task_KF.py

In [1]:
# import libraries
# make sure these directories are in the python path., 
from bmimultitasks import SimBMIControlMulti, SimBMICosEncKFDec
from features import SaveHDF
from features.simulation_features import get_enc_setup, SimKFDecoderRandom, SimCosineTunedEnc,SimIntentionLQRController
from riglib import experiment

import time
import numpy as np

/home/sijia-aw/BMi3D_my/lab_bmi3d/riglib/blackrock/cerelink.py:13: UserWarning: Unable to import cerebus library. Check if is installed if using the Blackrock NeuroPort system
  warnings.warn("Unable to import cerebus library. Check if is installed if using the Blackrock NeuroPort system")


##  set up trial information

In [2]:
#generate task params
N_TARGETS = 8
N_TRIALS = 6
seq = SimBMIControlMulti.sim_target_seq_generator_multi(
N_TARGETS, N_TRIALS)

##  simulation encoder decoder setup

In [3]:
#clda on random 
DECODER_MODE = 'trainedKF' # in this case we load simulation_features.SimKFDecoderRandom
ENCODER_TYPE = 'cosine_tuned_encoder'
LEARNER_TYPE = 'dumb' # to dumb or not dumb it is a question 'feedback'
UPDATER_TYPE = 'none' #none or "smooth_batch"


SAVE_HDF = False
DEBUG_FEATURE = False

#neuron set up : 'std (20 neurons)' or 'toy (4 neurons)' 
N_NEURONS, N_STATES, sim_C = get_enc_setup(sim_mode = 'toy')

features.simulation_features: get_enc_setup has a tuning_level of 1 



## from the setup options, set up experiment

In [4]:
# set up assist level
assist_level = (0, 0)

base_class = SimBMIControlMulti
feats = []

#set up intention feedbackcontroller
#this ideally set before the encoder
feats.append(SimIntentionLQRController)

#set up the encoder
if ENCODER_TYPE == 'cosine_tuned_encoder' :
    feats.append(SimCosineTunedEnc)
    print(f'{__name__}: selected SimCosineTunedEnc\n')
    
    
   #take care the decoder setup
if DECODER_MODE == 'random':
    feats.append(SimKFDecoderRandom)
    print(f'{__name__}: set base class ')
    print(f'{__name__}: selected SimKFDecoderRandom \n')
else: #defaul to a cosEnc and a pre-traind KF DEC
    from features.simulation_features import SimKFDecoderSup
    feats.append(SimKFDecoderSup)
    print(f'{__name__}: set decoder to SimKFDecoderSup\n')
    
    
#you know what? 
#learner only collects firing rates labeled with estimated estimates
#we would also need to use the labeled data
#to update the decoder.
if UPDATER_TYPE == 'smooth_batch':
    from features.simulation_features import SimSmoothBatch
    feats.append(SimSmoothBatch)
else: #defaut to none 
    print(f'{__name__}: need to specify an updater')
    
if DEBUG_FEATURE: 
    from features.simulation_features import DebugFeature
    feats.append(DebugFeature)
    
    
#save everthing in a kw
kwargs = dict()
kwargs['sim_C'] = sim_C

__main__: selected SimCosineTunedEnc

__main__: set decoder to SimKFDecoderSup

__main__: need to specify an updater


## make our experiment class

In [5]:
#spawn the task
Exp = experiment.make(base_class, feats=feats)
#print(Exp)
exp = Exp(seq, **kwargs)
exp.init()

metaclass constructor
(<class 'features.simulation_features.SimIntentionLQRController'>, <class 'features.simulation_features.SimCosineTunedEnc'>, <class 'features.simulation_features.SimKFDecoderSup'>, <class 'bmimultitasks.SimBMIControlMulti'>)
[<class 'features.simulation_features.SimIntentionLQRController'>, <class 'features.simulation_features.SimCosineTunedEnc'>, <class 'features.simulation_features.SimKFDecoderSup'>]

features.simulation_features.SimIntentionLQRController: LQRController used 

finished executing Experiment.__init__

SimCosineTunedEnc SSM: State space: ['hand_px', 'hand_py', 'hand_pz', 'hand_vx', 'hand_vy', 'hand_vz', 'offset'] 

neural encoder init function  <SimBMIControlMulti(Thread-4, initial)>
Creating simulation decoder..
SimKFDecoderSup:loaded self.encoder
<riglib.bmi.sim_neurons.GenericCosEnc object at 0x7f64462f8580> <class 'riglib.bmi.sim_neurons.GenericCosEnc'>
SimKFDecoderSup:loaded self.ssm
<riglib.bmi.sim_neurons.GenericCosEnc object at 0x7f64462f85

# now comes to the step through BMIloop


In [6]:
#riglib.experiment: line 597 - 601
exp.next_trial = next(exp.gen)
exp._parse_next_trial()

runseq


## decode neural features to move cursor
this section basically steps through the move_plant stuff

@riglib.bmi.BMILoop.move_plant

In [7]:
# bmi feature extraction, eh
#riglib.bmi: 1202
feature_data = exp.get_features()
feature_data

{'spike_counts': array([[0],
        [0],
        [0],
        [0]], dtype=uint32),
 'bin_edges': None}

In [8]:
# Determine the target_state and save to file
current_assist_level = exp.get_current_assist_level()
if np.any(current_assist_level > 0) or exp.learn_flag:
    target_state = exp.get_target_BMI_state(self.decoder.states)
else:
    target_state = np.ones([exp.decoder.n_states, exp.decoder.n_subbins]) * np.nan

In [13]:
#decode the new features
#riglib.bmi.bmiloop: line 1245
neural_features = feature_data[exp.extractor.feature_type]
tmp = exp.call_decoder(neural_features, target_state, **kwargs)

#saved as task data
exp.task_data['internal_decoder_state'] = tmp
tmp


array([[0.        ],
       [0.        ],
       [0.        ],
       [0.02883588],
       [0.        ],
       [0.01323624],
       [1.        ]])

In [14]:
# reset the plant position
# @riglib.bmi.BMILoop.move_plant  line:1254
exp.plant.drive(exp.decoder)
exp.plant.position

array([0., 0., 0.])

# try out some multiple inheritance technique

In [25]:
class A(object):
    def __init__(self):
        print("Entering A")
        print("Leaving A")
        
    def print_stuff(self):
        print('print stuff: A')
        

class B(object):
    def __init__(self):
        print("Entering B")
        super(B, self).__init__()
        print("Leaving B")
        
    def print_stuff(self):
        print('print stuff: B')
        super().print_stuff()
        


class C(B, A):
    def __init__(self):
        print("Entering C")
        super(C, self).__init__()
        print("Leaving C")
    
    def print_stuff(self):
        print('print stuff: C')
        super().print_stuff()
        
        
c = C()


super(C, c).print_stuff()


Entering C
Entering B
Entering A
Leaving A
Leaving B
Leaving C
print stuff: B
print stuff: A
